In [ ]:
import random
from abc import ABC, abstractmethod

# ANSI escape codes for colored output
RED = '\033[91m'
BLACK = '\033[90m'
RESET = '\033[0m'

# Abstraction
class Board(ABC):  # Board is an abstract base class
    def __init__(self, size):
        self._size = size
        self._board = self._create_board()  # Encapsulation

    @abstractmethod
    def _create_board(self):  # Abstract method
        pass

    def print_board(self, reveal_mines=False):
        # Print column headers
        print("   " + RED + " ".join(str(i+1) for i in range(self._size)) + RESET)
        for idx, row in enumerate(self._board):
            if reveal_mines:
                print(f"{RED}{idx+1:2}{RESET} " + " ".join(BLACK + cell + RESET if cell == '*' else cell for cell in row))
            else:
                print(f"{RED}{idx+1:2}{RESET} " + " ".join(cell if cell != '*' else ' ' for cell in row))

# Inheritance and Encapsulation
class MinesweeperBoard(Board):  # MinesweeperBoard inherits from Board
    def __init__(self, size, num_mines):
        super().__init__(size)
        self._num_mines = num_mines  # Encapsulation
        self._place_mines()  # Encapsulation

    def _create_board(self):
        return [[' ' for _ in range(self._size)] for _ in range(self._size)]

    def _place_mines(self):
        mines_placed = 0
        while mines_placed < self._num_mines:
            row = random.randint(0, self._size - 1)
            col = random.randint(0, self._size - 1)
            if self._board[row][col] != '*':
                self._board[row][col] = '*'
                mines_placed += 1

    def get_adjacent_mines(self, row, col):
        directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
        count = 0
        for dr, dc in directions:
            r, c = row + dr, col + dc
            if 0 <= r < self._size and 0 <= c < self._size and self._board[r][c] == '*':
                count += 1
        return count

# Factory Method
class BoardFactory:
    @staticmethod
    def create_board(board_type, size, num_mines=None):
        if board_type == "Minesweeper":
            return MinesweeperBoard(size, num_mines)
        else:
            raise ValueError(f"Unknown board type: {board_type}")

# Encapsulation and usage of the MinesweeperBoard class
class MinesweeperGame:
    _instance = None  # Singleton

    def __new__(cls, *args, **kwargs):
        if cls._instance is None:
            cls._instance = super(MinesweeperGame, cls).__new__(cls)
        return cls._instance

    def __init__(self, size, num_mines):
        if not hasattr(self, '_initialized'):
            self._board = BoardFactory.create_board("Minesweeper", size, num_mines)  # Using Factory Method
            self._play_board = [[' ' for _ in range(size)] for _ in range(size)]
            self._game_over = False
            self._revealed_cells = 0
            self._initialized = True

    def _reveal_cell(self, row, col):
        if self._play_board[row][col] != ' ':
            return

        if self._board._board[row][col] == '*':
            self._game_over = True
            return

        adjacent_mines = self._board.get_adjacent_mines(row, col)
        if adjacent_mines > 0:
            self._play_board[row][col] = str(adjacent_mines)
        else:
            self._play_board[row][col] = '0'
            directions = [(-1, -1), (-1, 0), (-1, 1), (0, -1), (0, 1), (1, -1), (1, 0), (1, 1)]
            for dr, dc in directions:
                r, c = row + dr, col + dc
                if 0 <= r < self._board._size and 0 <= c < self._board._size:
                    self._reveal_cell(r, c)

        self._revealed_cells += 1

    def _check_win_condition(self):
        total_cells = self._board._size * self._board._size
        if total_cells - self._revealed_cells == self._board._num_mines:
            self._game_over = True
            print("Congratulations! You've won the game!")
            self._print_play_board()

    def play(self):
        while not self._game_over:
            self._print_play_board()
            try:
                row, col = map(int, input("Enter row and column to reveal (e.g., '1 2'): ").split())
                row -= 1
                col -= 1
                if not (0 <= row < self._board._size and 0 <= col < self._board._size):
                    print("Invalid input. Please enter values within the board size.")
                    continue
                self._reveal_cell(row, col)
                if self._game_over:
                    if self._revealed_cells == (self._board._size * self._board._size - self._board._num_mines):
                        print("Congratulations! You've won the game!")
                    else:
                        print("Game Over! You hit a mine.")
                        self._board.print_board(reveal_mines=True)
                else:
                    self._check_win_condition()
                    if not self._game_over:
                        self._print_play_board()
            except ValueError:
                print("Invalid input. Please enter row and column as two integers separated by a space.")

        print("Thanks for playing!")

    def _print_play_board(self):
        print("   " + RED + " ".join(str(i+1) for i in range(self._board._size)) + RESET)
        for idx, row in enumerate(self._play_board):
            print(f"{RED}{idx+1:2}{RESET} " + " ".join(row))

if __name__ == "__main__":
    size = 8  # Board size
    num_mines = 10  # Number of mines
    game = MinesweeperGame(size, num_mines)
    game.play()

   1 2 3 4 5 6 7 8
 1                
 2                
 3                
 4                
 5                
 6                
 7                
 8                
Enter row and column to reveal (e.g., '1 2'): 1 1
   1 2 3 4 5 6 7 8
 1 1              
 2                
 3                
 4                
 5                
 6                
 7                
 8                
   1 2 3 4 5 6 7 8
 1 1              
 2                
 3                
 4                
 5                
 6                
 7                
 8                
Enter row and column to reveal (e.g., '1 2'): 1 2
Game Over! You hit a mine.
   1 2 3 4 5 6 7 8
 1   *            
 2     *          
 3                
 4         *   * *
 5                
 6         *   * *
 7                
 8 * *            
Thanks for playing!
